## Time series prediction

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor
from utils import MACD, RSI, Drawdown_current, volatility_rolling, extract 

In [ ]:
returns_all = pd.read_csv('230216_returns.csv')

In [3]:
# extract des retours les prix
# extract des retours les features

In [ ]:
hu1_comdty = extract(returns_all, 'HU1 Comdty')
hu1_comdty["Close"] = Close(hu1_comdty)

In [ ]:
hu1_comdty["MACD"] = MACD(hu1_comdty)
hu1_comdty["RSI"] = RSI(hu1_comdty)
hu1_comdty["Drawdown"] = Drawdown_current(hu1_comdty)
hu1_comdty["Volatility_20"] = volatility_rolling(hu1_comdty, window=20)
hu1_comdty["Volatility_60"] = volatility_rolling(hu1_comdty, window=60)

lags = list(range(1, 11))
for lag in lags:
    hu1_comdty[f'lag_{lag}'] = hu1_comdty['log_return'].shift(lag)